# Leeds Cultural Investment Programme - Processing Pipeline
This is the script that processes LCIP data ready for visualising on the site. This script is run as the data is updated on a quarterly basis.

#### Import the necessary libraries

In [1]:
import os 
import pandas as pd
from pathlib import Path

from thefuzz import process

ROOT = Path('../../')
ROOT.resolve()

PosixPath('/Users/tazminchiles/Documents/GitHub/lcc-culture-dashboard')

#### Define paths to the files we need

In [2]:
OUT_DIR = os.path.join(ROOT, 'src', '_data', 'viz', 'lcip')
DATA_DIR = os.path.join(ROOT, 'data', 'lcip')
WARD_DATA = os.path.join(ROOT, 'data', 'leeds_wards.csv')

LCIP_DATA = os.path.join(ROOT, 'data', 'lcip', 'LCIP GRANT STATS FINAL Q1 & 2 2024.xlsx')

#### Define our variables

In [3]:

ward_data = pd.read_csv(WARD_DATA)
lcip_data = pd.read_excel(LCIP_DATA, sheet_name=['Inspire 2024', 'Grow Project 2024', 'Activate 2024', 'Grow Revenue 2024', 'Thrive 2024', 'Cultural Anchors 2024'])
project_data = pd.read_excel(LCIP_DATA, sheet_name=['Inspire 2024', 'Grow Project 2024', 'Activate 2024'])
revenue_data = pd.read_excel(LCIP_DATA, sheet_name=['Grow Revenue 2024', 'Thrive 2024', 'Cultural Anchors 2024'])

diversity_metrics = [
    'AGE - APPLIED',
    'DISABILITY - APPLIED',
    'SEX - APPLIED',
    'GENDER REGISTERED AT BIRTH - APPLIED',
    'ETHNIC ORIGIN - APPLIED',
    'SEXUAL ORIENTATION - APPLIED',
    'RELIGIOUS BELIEF - APPLIED',
    'CARER - APPLIED',
    'AGE - FUNDED',
    'DISABILITY - FUNDED',
    'SEX - FUNDED',
    'GENDER REGISTERED AT BIRTH - FUNDED',
    'ETHNIC ORIGIN - FUNDED',
    'SEXUAL ORIENTATION - FUNDED',
    'RELIGIOUS BELIEF - FUNDED',
    'CARER - FUNDED'
]

ward_themes = [
    'WARDS - APPLICANT BASED', 
    'WARDS - FUNDED',
    'WARDS - RECEIVING ACTIVITY'
]

project_names = ['Inspire', 'Grow Project', 'Activate']
revenue_names = ['Grow Revenue', 'Thrive', 'Cultural Anchors']


A function for fuzzy matching between datasets

In [4]:
def fuzzy_merge(df1, df2, key1, key2, threshold=90):
    s = df2[key2].tolist()

    m = df1[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))
    df1['matches'] = m.apply(lambda x: x[0] if x else None)
    df1['score'] = m.apply(lambda x: x[1] if x else None)

    merged = pd.merge(df1, df2, left_on='matches', right_on=key2, how='left')
    merged.drop(['matches', 'score'], axis=1, inplace=True)
    return merged

### Process ward data

In [5]:

def process_wards(ward_data, data, out_path, theme, output_file):
    wards_data = data[data['THEME'] == theme]
    merged_data = (
        fuzzy_merge(ward_data, wards_data, 'WD21NM', 'METRIC')
        .dropna()
        .drop(columns='WD21NM')
        .rename(columns={'WD21CD': 'ward_code', 'THEME': 'metric', 'METRIC': 'ward_name', 'R1 Q1': 'value'})
    )
    merged_data['value'] = merged_data['value'].round(0).astype(int)
    merged_data.to_csv(os.path.join(out_path, output_file), index=False)

### Process EDI data 

To preserve anonymity of applicants, EDI data will be aggregated across Project and Revenue strands. This function extracts diversity metrics from the LCIP grants dataset, and combines them into a single dataset for visualisation on the dashboard.

In [6]:
def process_diversity_metrics(data,filename_stem, project):
    out_path = os.path.join(OUT_DIR, 'diversity', project)

    # Group together applied and funded metrics
    applied = data[data['THEME'].str.contains(r' - APPLIED')]
    funded = data[data['THEME'].str.contains(r' - FUNDED')]
    applied.loc[:, 'THEME'] = applied['THEME'].str.replace(r' - APPLIED', '', regex=False)
    funded.loc[:, 'THEME'] = funded['THEME'].str.replace(r' - FUNDED', '', regex=False)

    # Loop through each theme
    unique_themes = applied['THEME'].unique()
    for theme in unique_themes:
        applied_theme = applied[applied['THEME'] == theme].copy()
        funded_theme = funded[funded['THEME'] == theme].copy()

        applied_theme['TOTAL'] = applied_theme['R1 Q1'] + applied_theme['R2 Q2']
        funded_theme['TOTAL'] = funded_theme['R1 Q1'] + funded_theme['R2 Q2']

        applied_theme = applied_theme[['THEME', 'METRIC', 'TOTAL']].copy()
        funded_theme = funded_theme[['THEME', 'METRIC', 'TOTAL']].copy()

        diversity_theme = pd.merge(
            applied_theme,
            funded_theme,
            on=['THEME', 'METRIC'],
            how='outer',
            suffixes=('_APPLIED', '_FUNDED')
        )

        diversity_theme['TOTAL_APPLIED'] = diversity_theme['TOTAL_APPLIED'].round(0).astype('Int64')
        diversity_theme['TOTAL_FUNDED'] = diversity_theme['TOTAL_FUNDED'].round(0).astype('Int64')

        diversity_theme.drop(columns=['THEME'], inplace=True)


        diversity_theme = diversity_theme.rename(columns={
            'TOTAL_APPLIED': 'APPLIED',
            'TOTAL_FUNDED': 'FUNDED'
        })

        diversity_theme = diversity_theme.dropna()

        theme_filename = theme.replace(" ", "_").replace("/", "_").replace('(', '').replace(')', '').replace('_-_', '_').lower() + '.csv'
        theme_out_path = os.path.join(out_path, 'diversity', filename_stem, theme_filename)

        print(diversity_theme)

        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        if not diversity_theme.empty:
            diversity_theme.to_csv(out_path, index=False)

In [7]:
def name_cleanup(name):
    name_clean = name.replace(' 2024', '')
    name_clean = (name_clean.lower().strip().replace(' ', '_'))
    return name_clean

In [8]:
def sheet_cleanup(sheet):
    sheet = sheet.dropna(axis='columns', how='all')
    sheet = sheet.copy()
    sheet['THEME'] = sheet['THEME'].str.rstrip()
    sheet['METRIC'] = sheet['METRIC'].str.rstrip()
    return sheet

#### Process Project grants data

In [9]:
project_diversity_data = pd.DataFrame()

# Process project grants
for name, sheet in project_data.items():
    # Define output path
    theme_path = os.path.join(ROOT, OUT_DIR, name)
    os.makedirs(theme_path, exist_ok=True)

    # Clean up sheet names
    name = name_cleanup(name)

    # Clean up data
    sheet = sheet_cleanup(sheet)

    # Create datasets for each theme:
    themes = sheet['THEME'].unique()

    project_totals = pd.DataFrame()

    # Loop through each theme in the sheet
    for theme in themes:
        if theme in diversity_metrics:
            process_diversity_metrics(sheet, name, project='project')





    #         theme_df = sheet[sheet['THEME'] == theme].copy()
    #         theme_df = theme_df.fillna(0)
    #         theme_df[f'total_{name}'] = theme_df.iloc[:, 2] + theme_df.iloc[:, 3]
    #         project_totals[f'total_{name}'] = theme_df.loc[theme_df[f'total_{name}']]
    
    # print(project_totals)


                               METRIC  APPLIED  FUNDED
0                              Circus        0       0
1                       Combined Arts        5       0
2                              Crafts        1       1
3                               Dance        1       0
4                              Design        0       0
5                       Digital Media        2       2
6                   Festival/Carnival        3       1
7                                Film        4       1
8   Literature / Poetry / Spoken Word        2       0
9                            Live Art        0       0
10                              Music       11       3
11                        Photography        2       1
12                            Theatre        4       1
13                         Visual Art        8       6


IsADirectoryError: [Errno 21] Is a directory: '../../src/_data/viz/lcip/diversity/project'

In [ ]:
for name, sheet in lcip_data.items():

    # Define output path
    theme_path = os.path.join(OUT_DIR, name)
    os.makedirs(theme_path, exist_ok=True)
    diversity_path = os.path.join(OUT_DIR, 'diversity')
    os.makedirs(diversity_path, exist_ok=True)

    # Clean up sheet names
    name = name_cleanup(name)

    # Clean up data
    sheet = sheet_cleanup(sheet)

    # Create datasets for each theme:
    themes = sheet['THEME'].unique()

    if name in project_data: 
        for theme in themes:
            if theme in diversity_metrics:
                process_diversity_metrics(sheet, os.path.join(diversity_path, 'project'))
            else: 
                print('not diversity')
            # elif theme in ward_themes:
            #     process_wards(ward_data, sheet, theme_path, 'WARDS - APPLICANT BASED', 'applications_by_ward.csv')
            #     process_wards(ward_data, sheet, theme_path, 'WARDS - RECEIVING ACTIVITY', 'received_by_ward.csv')
            #     process_wards(ward_data, sheet, theme_path, 'WARDS - FUNDED', 'funded_by_ward.csv')
            # else:
            #     theme_df = sheet[sheet['THEME'] == theme]
            #     theme_df = theme_df.pivot_table(index='THEME', columns='METRIC', values=['PERIOD_1'])
            #     theme_df = theme_df.round(0).astype(int)
            #     # theme_df.columns = theme_df.columns.str.replace(',',' ')
            #     theme_filename = theme.replace(" ", "_").replace("/", "_").replace('(', '').replace(')', '').replace('_-_', '_').lower() + '.csv'
            #     theme_df.to_csv(os.path.join(theme_path, name), index=True)



not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity
not diversity


### Test code

In [ ]:
for name, sheet in lcip_data.items():
    name = name.replace(' 2024', '')
    name = (name.lower().strip().replace(' ', '_'))
    out_path = os.path.join(OUT_DIR, name)
    os.makedirs(out_path, exist_ok=True)

    # Drop empty columns
    sheet = sheet.dropna(axis='columns', how='all')

    # Clean up the THEME and METRIC columns
    sheet['THEME'] = sheet['THEME'].str.rstrip()
    sheet['METRIC'] = sheet['METRIC'].str.rstrip()

    # Create datasets for each theme:
    themes = sheet['THEME'].unique()
    for theme in themes:
        if theme in diversity_metrics:
            process_diversity_metrics(sheet, OUT_DIR, name)
        elif theme in ward_themes:
            process_wards(ward_data, sheet, out_path, 'WARDS - APPLICANT BASED', 'applications_by_ward.csv')
            process_wards(ward_data, sheet, out_path, 'WARDS - RECEIVING ACTIVITY', 'received_by_ward.csv')
            process_wards(ward_data, sheet, out_path, 'WARDS - FUNDED', 'funded_by_ward.csv')
        else:
            theme_df = sheet[sheet['THEME'] == theme]
            theme_df = theme_df.pivot_table(index='THEME', columns='METRIC', values=['PERIOD_1'])
            theme_df = theme_df.round(0).astype(int)
            # theme_df.columns = theme_df.columns.str.replace(',',' ')
            theme_filename = theme.replace(" ", "_").replace("/", "_").replace('(', '').replace(')', '').replace('_-_', '_').lower() + '.csv'
            theme_df.to_csv(os.path.join(out_path, name), index=True)


/var/folders/9k/g5s5mvqs2_jb7ftn5m2jyqs00000gn/T/ipykernel_3978/3876376796.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet['THEME'] = sheet['THEME'].str.rstrip()
/var/folders/9k/g5s5mvqs2_jb7ftn5m2jyqs00000gn/T/ipykernel_3978/3876376796.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet['METRIC'] = sheet['METRIC'].str.rstrip()


KeyError: 'PERIOD_1'

In [ ]:
for name, sheet in lcip_data.items():
    name = name.replace(' 2024', '')
    out_path = os.path.join(OUT_DIR, name)
    os.makedirs(out_path, exist_ok=True)

    # Drop empty columns
    sheet = sheet.dropna(axis='columns', how='all')

    if name.isin(project_data):
        process_project_data(sheet, name)
    else:
        process_revenue_data(sheet, name)
    
    # Replace column names so that we can refer to them later
    if sheet.shape[1] == 5:
        sheet.columns = ['THEME', 'METRIC', 'PERIOD_1', 'PERIOD_2', 'PERIOD_3']
    if sheet.shape[1] == 4:
        sheet.columns = ['THEME', 'METRIC', 'PERIOD_1', 'PERIOD_2']
    elif sheet.shape[1] == 3:
        sheet.columns = ['THEME', 'METRIC', 'PERIOD_1']

    # Clean up the THEME and METRIC columns
    sheet['THEME'] = sheet['THEME'].str.rstrip()
    sheet['METRIC'] = sheet['METRIC'].str.rstrip()

    # Loop through each unique item in the THEME column
    themes = sheet['THEME'].unique()
    for theme in themes:
        theme_df = sheet[sheet['THEME'] == theme]

        if theme in diversity_metrics:
            # Apply specific processing for diversity metrics
            process_diversity_metrics(theme_df, OUT_DIR, name)
        else:
            # Create a pivot table for each non-diversity theme
            theme_pivot = theme_df.pivot_table(index='THEME', columns='METRIC', values=['PERIOD_1'], aggfunc='first')
            theme_pivot = theme_pivot.round(0).astype(int)

            # Generate a filename from the theme name
            theme_filename = theme.replace(" ", "_").replace("/", "_").replace('(', '').replace(')', '').replace('_-_', '_').lower() + '.csv'
            
            # Save the pivot table to a CSV file
            theme_pivot.to_csv(os.path.join(out_path, theme_filename), index=True)

AttributeError: 'str' object has no attribute 'isin'

In [ ]:
for name, sheet in lcip_data.items():
    name = str.replace(name, ' 2024', '')
    file_path = os.path.join(DATA_DIR, name)
    out_path = os.path.join(OUT_DIR, name)

    # Rename column headings, clean up themes and delete empty columns
    sheet.columns = ['THEME', 'METRIC', 'PERIOD_1', 'PERIOD_2', 'PERIOD_3', 'PERIOD_4']
    sheet = sheet.dropna(axis='columns', how='all')
    sheet['THEME'] = sheet['THEME'].str.rstrip()
    sheet['METRIC'] = sheet['METRIC'].str.rstrip()

    
    themes = sheet['THEME'].unique()
    for theme in themes:
        theme_df = sheet[sheet['THEME'] == theme]
        theme_df = theme_df.pivot_table(index='THEME', columns='METRIC', values=['PERIOD_1'])
        theme_df = theme_df.round(0).astype(int)
        # theme_df.columns = theme_df.columns.str.replace(',',' ')
        theme_filename = theme.replace(" ", "_").replace("/", "_").replace('(', '').replace(')', '').replace('_-_', '_').lower() + '.csv'
        theme_df.to_csv(os.path.join(out_path, name), index=True)

    if sheet['THEME'].isin(diversity_metrics).any():
        process_diversity_metrics(sheet, OUT_DIR, name)

    process_wards(ward_data, sheet, out_path, 'WARDS - APPLICANT BASED', 'applications_by_ward.csv')
    process_wards(ward_data, sheet, out_path, 'WARDS - RECEIVING ACTIVITY', 'received_by_ward.csv')


    

/var/folders/9k/g5s5mvqs2_jb7ftn5m2jyqs00000gn/T/ipykernel_1659/2962489398.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet['THEME'] = sheet['THEME'].str.rstrip()
/var/folders/9k/g5s5mvqs2_jb7ftn5m2jyqs00000gn/T/ipykernel_1659/2962489398.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sheet['METRIC'] = sheet['METRIC'].str.rstrip()


KeyError: "['R1 Q1'] not in index"